In [58]:
#import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#import html/scraping libraries
import requests
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize

#import mapping libraries
import geopy
import folium
from geopy.geocoders import Nominatim
from IPython.display import Image

#import colors
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#download portland geojson
!wget -q -O 'pdx_data.geojson' https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson

In [291]:
with open('pdx_data.geojson') as f:
    pdx_geo = json.load(f)

We can look at what kind of data we have:

In [261]:
pdx_geo['features'][4]

{'type': 'Feature',
 'properties': {'OBJECTID': 5,
  'NAME': 'CULLY ASSOCIATION OF NEIGHBORS',
  'COMMPLAN': ' ',
  'SHARED': ' ',
  'COALIT': 'CNN',
  'HORZ_VERT': 'HORZ',
  'MAPLABEL': 'Cully Association of Neighbors',
  'ID': 23,
  'Shape_Length': 18179.392090202025,
  'Shape_Area': 16580624.669252962},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.6147122245439, 45.54829038307215],
    [-122.60785446051207, 45.548264575609245],
    [-122.59938743920893, 45.54822488960527],
    [-122.59938092282987, 45.54857652975009],
    [-122.59937739514577, 45.54885060900733],
    [-122.59937020323359, 45.54928954232259],
    [-122.59936472979857, 45.54956382821686],
    [-122.5993575315982, 45.550002589256856],
    [-122.59935401020228, 45.550276840846834],
    [-122.59934741746642, 45.55062642430259],
    [-122.59934328880935, 45.550989851980546],
    [-122.59933472966135, 45.55149739922728],
    [-122.59933165023655, 45.551730830835176],
    [-122.5993316547281, 45.55246988194597]

We need additional data for this project, I have downloaded data from portlandmap.com also, I am scraping data from 2019 from a site that seems to have the most up to date data for population statistics by neighborhood

In [34]:
permits_df = pd.read_csv('Residential_Building_Permits.csv')
#lets look at the time frame of the permits
print('Date Range : {} to {}'.format(min(permits_df['YEAR_']),max(permits_df['YEAR_'])))
print('Size of the data: {}'.format(permits_df.shape))

Date Range : 1994 to 2019
Size of the data: (30106, 28)


Lets narrow the data down to 2016-present and see what kind of permits we are dealing with

In [41]:
permits_df = permits_df[permits_df['YEAR_'] >= 2016]
print(permits_df.shape)
permits_df.NEWCLASS.unique()

(5527, 28)


array(['New Construction', 'Alteration', 'Move', 'Addition',
       'Replacement', 'First'], dtype=object)

With the objective of estimating new residents, we will only keep the 'Addition' and "New Construction" values.

In [50]:
permits_df = permits_df[permits_df['NEWCLASS'].isin(['New Construction','Addition'])]
permits_df.shape


(4689, 28)

In [51]:
permits_df.head()

,X,Y,OBJECTID,INDATE,ISSUEDATE,STATUS,YEAR_,NEWCLASS,NEWTYPE,NBRHOOD,...,NEW_UNITS,FOLDER_DES,VALUATION,CONST,PROPLOT,PROPGISID1,PROPERTYRO,FOLDERRSN,X_COORD,Y_COORD
23336,-122.771014,45.720797,23337,2013-03-06T00:00:00.000Z,2016-07-15T15:39:49.000Z,Admin Hold,2016,New Construction,Single Family Dwelling,NaN,...,1,NEW SINGLE FAMILY RESIDENCE/2 STORY/2 CAR GARA...,912966,V-B,R502908,3N1W23 700,3N1W23 00700,3283314,7.622136e+06,757041.415376
23338,-122.588079,45.562411,23339,2013-04-05T00:00:00.000Z,2017-06-16T09:49:52.000Z,Issued,2017,New Construction,Apartments/Condos,CULLY ASSOCIATION OF NEIGHBORS,...,12,NEW 12 UNIT THREE STORY APARTMENT BUILDING AND...,1112998,V-B,R209993,1N2E20AB 4900,1N2E20AB 04900,3296117,7.667349e+06,698049.685919
23339,-122.588340,45.562319,23340,2013-04-05T00:00:00.000Z,2017-06-16T09:50:06.000Z,Issued,2017,New Construction,Apartments/Condos,CULLY ASSOCIATION OF NEIGHBORS,...,12,"NEW 12 UNIT THREE STORY APARTMENT BUILDING, me...",1112998,V-B,R209993,1N2E20AB 4900,1N2E20AB 04900,3296123,7.667282e+06,698017.793594
23344,-122.568927,45.527362,23345,2013-06-28T00:00:00.000Z,2017-03-17T08:22:36.000Z,Final,2017,New Construction,Townhouse,MONTAVILLA,...,1,UNIT 1 OF 5 UNIT TOWHOUSE - ALL ON SAME TAX LO...,200112,V-B,R498064,1N2E33BD 2901,1N2E33BD 02901,3333750,7.671926e+06,685148.493545
23345,-122.568927,45.527281,23346,2013-06-28T00:00:00.000Z,2017-03-14T08:32:13.000Z,Final,2017,New Construction,Townhouse,MONTAVILLA,...,1,UNIT 2 OF 5 UNIT TOWHOUSE - ALL ON SAME TAX LO...,210687,V-B,R498064,1N2E33BD 2901,1N2E33BD 02901,3333757,7.671925e+06,685118.964405


Now we need to keep only entries that have a neighborhood assignment

In [57]:
permits_df = permits_df[permits_df['NBRHOOD'].notnull()]
permits_df.shape,len(permits_df.NBRHOOD.unique())

((4663, 28), 105)

It looks like we have 105 unique neighborhoods, with 4663 building permits in identified neighborhoods.

Lets continue to get some more data.


In [59]:
pdxdata_url = 'https://www.pdxmonthly.com/home-and-real-estate/2019/03/portland-neighborhoods-by-the-numbers-2019-the-city'
pdx_soup = BeautifulSoup(requests.get(pdxdata_url).text,'lxml')

In [63]:
#get a list of columns
col_names = [i.text for i in pdx_soup.find_all('th')]
#extraxt each datapoint for each row in the table
by_hood = [[entry.text for entry in row.find_all('td')]for row in pdx_soup.find_all('tr')[1:]]

In [85]:
pdx_hood_df = pd.DataFrame(columns=col_names, data=by_hood)

In [143]:
pdx_hood_df[pdx_hood_df['Neighborhood'] == 'PORTLAND TOTAL**']

,Neighborhood,Average home sale price ($),Median home sale price ($),Average cost per square foot ($),Days on market (avg.),Homes sold in 2018 (#),Condo sales (%),1-year median price change (2017–2018) (%),5-year median price change (2014–2018) (%),Distressed property sales (%),...,Commute by public transit (%),Commute by bike (%),Commute by walking (%),Commute by biking and walking (%),Number of Max/streetcar/tram lines,Number of TriMet bus lines,Number of transit lines (bus/MAX/streetcar/tram),"Miles of bike routes (lanes, boulevards, multiuse paths)",Miles of bike routes (all) per square mile,Walk score
95,PORTLAND TOTAL**,"480,268","420,000",255,37,"9,427",17%,5%,35%,1%,...,12.3,6.5,5.7,12.2,8,57,79,469,3.2,65


In [147]:
pdx_hood_df.drop(pdx_hood_df.index[95], inplace=True)
print(pdx_hood_df.shape)

pdx_hood_df.head()

(95, 57)


,Neighborhood,Average home sale price ($),Median home sale price ($),Average cost per square foot ($),Days on market (avg.),Homes sold in 2018 (#),Condo sales (%),1-year median price change (2017–2018) (%),5-year median price change (2014–2018) (%),Distressed property sales (%),...,Commute by public transit (%),Commute by bike (%),Commute by walking (%),Commute by biking and walking (%),Number of Max/streetcar/tram lines,Number of TriMet bus lines,Number of transit lines (bus/MAX/streetcar/tram),"Miles of bike routes (lanes, boulevards, multiuse paths)",Miles of bike routes (all) per square mile,Walk score
0,ALAMEDA,"785,713","738,000",268,31,81,0%,8%,26%,1%,...,8.8,3.9,11,14.9,0,3,3,1,1.2,65
1,ARBOR LODGE,"465,743","459,450",252,28,132,13%,1%,33%,0%,...,14.2,3.6,11.9,15.6,1,4,5,6,7.2,72
2,ARDENWALD-JOHNSON CREEK,"418,640","434,500",185,89,10,0%,-7%,34%,10%,...,9.3,0.4,7.3,7.7,1,5,6,3,7.8,54
3,ARGAY,"391,146","399,750",168,26,79,5%,11%,52%,0%,...,8.6,15.8,0.3,16.1,0,5,5,6,2.9,45
4,ARLINGTON HEIGHTS,"1,012,913","862,500",306,49,20,0%,-6%,9%,0%,...,6,10.6,4.1,14.8,2,4,6,1,1.6,40


Now we have three dataframes with pertinent data for the project, we now need to combine the dataframes in a meaningful way with the data we want to keep

In [148]:
#check the nieghborhood number in each df
print("the geo file contains {} neighborhoods \nthe permit file contains {} neighborhoods \nthe neighborhood df contains {} neighborhoods".format(len(pdx_geo['features']),len(permits_df.NBRHOOD.unique()),len(pdx_hood_df['Neighborhood'])))

the geo file contains 98 neighborhoods 
the permit file contains 105 neighborhoods 
the neighborhood df contains 95 neighborhoods


looks like we need to reconcile these differences before moving forward. We will start with the permit file and compare it to the geofile

In [149]:
geo_hoods = [pdx_geo['features'][i]['properties']['MAPLABEL'].lower() for i in range(len(pdx_geo['features']))]
permit_hoods = [i.lower() for i in permits_df.NBRHOOD.unique()]

In [150]:
#here we will use built in python 'difference' function for the 'set' datatype
set(permit_hoods).difference(set(geo_hoods))

{'alameda/beaumont-wilshire',
 'alameda/irvington community assn.',
 'ardenwald-johnson creek/woodstock',
 'boise/eliot',
 'bridlemile/southwest hills residential league',
 'centennial community assn./pleasant valley',
 'eastmoreland/reed',
 'goose hollow foothills league/southwest hills residential league',
 'grant park/hollywood',
 'healy heights/southwest hills residential league',
 'hillside/northwest district assn.',
 'lents/powellhurst-gilbert',
 'pleasant valley/powellhurst-gilbert',
 'sylvan-highlands/southwest hills residential league'}

In [151]:
set(geo_hoods).difference(set(permit_hoods))

{'healy heights',
 'mc unclaimed #11',
 'mc unclaimed #13',
 'mc unclaimed #14',
 'mc unclaimed #5',
 'sunderland association of neighbors',
 'woodland park'}

We see that some permits are in neighborhoods that may over lap, however, we want to compare uniqueness, so we can separate each neighborhood to see if it produces a unique entry

In [152]:
ph_0 = [i.split('/')[0] for i in permit_hoods if '/' in i]
ph_1 = [i.split('/')[1] for i in permit_hoods if '/' in i]


In [153]:
permit_sep

['pleasant valley',
 'hillside',
 'bridlemile',
 'healy heights',
 'lents',
 'alameda',
 'sylvan-highlands',
 'eastmoreland',
 'ardenwald-johnson creek',
 'boise',
 'goose hollow foothills league',
 'grant park',
 'alameda',
 'centennial community assn.',
 'powellhurst-gilbert',
 'northwest district assn.',
 'southwest hills residential league',
 'southwest hills residential league',
 'powellhurst-gilbert',
 'beaumont-wilshire',
 'southwest hills residential league',
 'reed',
 'woodstock',
 'eliot',
 'southwest hills residential league',
 'hollywood',
 'irvington community assn.',
 'pleasant valley']

In [154]:
permit_sep = ph_0+ph_1
set(permit_sep).difference(set(permit_hoods))

{'centennial community assn.',
 'healy heights',
 'irvington community assn.',
 'northwest district assn.'}

In [155]:
permit_hoods[21]

'goose hollow foothills league'

In [156]:
'healy hieghts' in permit_hoods

False

We can see here that we have decisions to make on how we will merge the data.
Current Issues:
 1. the geo file contains unclaimed areas
 2. the permit file has combined neighborhoods
 3. the neighborhood file has fewer neighborhoods than the others <br><br>
Chosen Solution:
     - drop unclaimed neighborhoods from geo_file
     - keep first entry from permits
     - filter overall neighborhoods by neighborhood file (for completeness)<br><br>
Note: Its ok that the permit file is missing data, afterall it is not a requirement for new housing to go up in a certain area

The differences between the neighborhood file and geo_file need to be examined

In [157]:
n_hoods = [i.lower() for i in pdx_hood_df['Neighborhood']]

In [158]:
set(geo_hoods).difference(set(n_hoods))

{'argay terrace',
 'brentwood-darlington',
 'brooklyn action corps',
 'buckman community association',
 'centennial community association',
 'cully association of neighbors',
 'goose hollow foothills league',
 'hayden island neighborhood network',
 'hosford-abernethy neighborhood district assn.',
 'irvington community association',
 'lloyd district community association',
 'mc unclaimed #11',
 'mc unclaimed #13',
 'mc unclaimed #14',
 'mc unclaimed #5',
 'mt. scott-arleta',
 'mt. tabor',
 'northwest district association',
 'old town community association',
 'parkrose heights association of neighbors',
 'pearl district',
 'portland downtown',
 'sabin community association',
 'sellwood-moreland improvement league',
 'southwest hills residential league',
 'sumner association of neighbors',
 'sunderland association of neighbors',
 'wilkes community group'}

In [159]:
set(n_hoods).difference(set(geo_hoods))

{'argay',
 'brentwood/ darlington',
 'brooklyn',
 'buckman',
 'centennial',
 'cully',
 'downtown',
 'dunthorpe',
 'goose hollow',
 'hayden island',
 'hosford-abernethy',
 'irvington',
 'lloyd district',
 'mt scott-arleta',
 'mt tabor',
 'northwest district',
 'old town/ chinatown',
 'parkrose heights',
 'pearl',
 'sabin',
 'sellwood-moreland',
 'southwest hills',
 'sumner',
 'sunderland',
 'wilkes'}

visually, it can be reconcilled (with maps) that mc unclaimed 11 is donthrope, and 14 is northwest industrial.
To minimize the amount of manual correction, it would be best to update the neighborhood df to the standards of the permit/geo file, then correct the other few in the geofile(donthrope, nw industrial)

In [168]:
before = [i for i in n_hoods if i not in geo_hoods]
after = ['argay terrace','brentwood-darlington','brooklyn action corps','buckman community association',
         'centennial community association','cully association of neighbors','portland downtown',
         'dunthorpe','goose hollow foothills league','hayden island neighborhood network',
         'hosford-abernethy neighborhood district assn.','irvington community association',
         'lloyd district community association','mt. scott-arleta','mt. tabor','northwest district association',
         'old town community association','parkrose heights association of neighbors','pearl district',
         'sabin community association','sellwood-moreland improvement league','southwest hills residential league',
         'sumner association of neighbors','sunderland association of neighbors', 'wilkes community group']
replace_d = {k:v for k,v, in zip(before,after)}

In [169]:
replace_d

{'argay': 'argay terrace',
 'brentwood/ darlington': 'brentwood-darlington',
 'brooklyn': 'brooklyn action corps',
 'buckman': 'buckman community association',
 'centennial': 'centennial community association',
 'cully': 'cully association of neighbors',
 'downtown': 'portland downtown',
 'dunthorpe': 'dunthorpe',
 'goose hollow': 'goose hollow foothills league',
 'hayden island': 'hayden island neighborhood network',
 'hosford-abernethy': 'hosford-abernethy neighborhood district assn.',
 'irvington': 'irvington community association',
 'lloyd district': 'lloyd district community association',
 'mt scott-arleta': 'mt. scott-arleta',
 'mt tabor': 'mt. tabor',
 'northwest district': 'northwest district association',
 'old town/ chinatown': 'old town community association',
 'parkrose heights': 'parkrose heights association of neighbors',
 'pearl': 'pearl district',
 'sabin': 'sabin community association',
 'sellwood-moreland': 'sellwood-moreland improvement league',
 'southwest hills': '

In [177]:
#the replace dictionary looks good lets update the names
old_names = [i.lower() for i in pdx_hood_df['Neighborhood']]
new_names = [replace_d[i].upper() if i in replace_d.keys() else i.upper() for i in old_names]


In [178]:
new_names[:10]

['ALAMEDA',
 'ARBOR LODGE',
 'ARDENWALD-JOHNSON CREEK',
 'ARGAY TERRACE',
 'ARLINGTON HEIGHTS',
 'ARNOLD CREEK',
 'ASHCREEK',
 'BEAUMONT-WILSHIRE',
 'BOISE',
 'BRENTWOOD-DARLINGTON']

In [180]:
#set neighborhood column back in df
pdx_hood_df['Neighborhood'] = new_names

In [182]:
pdx_hood_df.to_csv('pdx_hoods.csv',index=False)
pdx_hood_df.head()


,Neighborhood,Average home sale price ($),Median home sale price ($),Average cost per square foot ($),Days on market (avg.),Homes sold in 2018 (#),Condo sales (%),1-year median price change (2017–2018) (%),5-year median price change (2014–2018) (%),Distressed property sales (%),...,Commute by public transit (%),Commute by bike (%),Commute by walking (%),Commute by biking and walking (%),Number of Max/streetcar/tram lines,Number of TriMet bus lines,Number of transit lines (bus/MAX/streetcar/tram),"Miles of bike routes (lanes, boulevards, multiuse paths)",Miles of bike routes (all) per square mile,Walk score
0,ALAMEDA,"785,713","738,000",268,31,81,0%,8%,26%,1%,...,8.8,3.9,11,14.9,0,3,3,1,1.2,65
1,ARBOR LODGE,"465,743","459,450",252,28,132,13%,1%,33%,0%,...,14.2,3.6,11.9,15.6,1,4,5,6,7.2,72
2,ARDENWALD-JOHNSON CREEK,"418,640","434,500",185,89,10,0%,-7%,34%,10%,...,9.3,0.4,7.3,7.7,1,5,6,3,7.8,54
3,ARGAY TERRACE,"391,146","399,750",168,26,79,5%,11%,52%,0%,...,8.6,15.8,0.3,16.1,0,5,5,6,2.9,45
4,ARLINGTON HEIGHTS,"1,012,913","862,500",306,49,20,0%,-6%,9%,0%,...,6,10.6,4.1,14.8,2,4,6,1,1.6,40


Now lets cleanup the permits neighborhood column

In [183]:
comb_list = [i.split('/')[0] if '/' in i else i for i in permits_df.NBRHOOD ]

In [187]:
permits_df['NBRHOOD'] = comb_list

Now, lets re-check to see the differences in the permits_df neighborhoods and pdx_hood_df neighborhoods

In [229]:
set(pdx_hood_df['Neighborhood']).difference(set(permits_df['NBRHOOD']))

{'DUNTHORPE', 'SUNDERLAND ASSOCIATION OF NEIGHBORS', 'WOODLAND PARK'}

In [240]:
geo_missing = list(set([i.upper() for i in geo_hoods]).difference(set(pdx_hood_df['Neighborhood'])))

In [292]:
#we update the geo_json file, we need to find out the place of the unknown neighborhoods
missing_idx = {}
for i in geo_missing:
    for j in range(len(pdx_geo['features'])):        
        if pdx_geo['features'][j]['properties']['NAME'] == i:
            missing_idx[i] = j

In [293]:
missing_idx

{'MC UNCLAIMED #14': 53,
 'MC UNCLAIMED #5': 17,
 'MC UNCLAIMED #13': 49,
 'MC UNCLAIMED #11': 44}

In [294]:
#we want to remove unclaimed 14, 5, 13 and rename 11
#we pop in order of descending idx so as to not change the position
print('Length before {}'.format(len(pdx_geo['features'])))
pdx_geo['features'][44]['properties']['NAME'] = 'DUNTHORPE'
pdx_geo['features'].pop(53)
pdx_geo['features'].pop(49)
pdx_geo['features'].pop(17)
print('Length after {}'.format(len(pdx_geo['features'])))

Length before 98
Length after 95


In [295]:
geo_hoods = [pdx_geo['features'][i]['properties']['NAME'] for i in range(len(pdx_geo['features']))]
set(geo_hoods).difference(set(pdx_hood_df['Neighborhood']))

set()

In [199]:
#Lets get the stat from the permits df that we want
p_grouped =permits_df.groupby('NBRHOOD')['NEW_UNITS'].sum().reset_index()
p_grouped.columns=['Neighborhood', 'New Units']

In [203]:
p_grouped.dtypes

Neighborhood    object
New Units        int64
dtype: object

In [224]:
new_units = []
for i in pdx_hood_df['Neighborhood']:
    if i in list(p_grouped['Neighborhood']):
        new_units.append(int(p_grouped[p_grouped['Neighborhood']==i]['New Units']))
    else:
        new_units.append(0)

In [227]:
pdx_hood_df['New Units'] = new_units

## Recap

Up to now, we have a cleaned Portland Geo_json file, extrated what we wanted from the building permit dataset, we have a dataset of portland by neighboorhood with the population, pop density, and projected new units

In [23]:
#get geocoords for portland
address = "Portland, OR"

geolocator = Nominatim(user_agent="extra_lime")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Portland, OR are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Portland, OR are 45.5202471, -122.6741949.


In [312]:
pdx_hood_df['Adjusted population'] = [int(i.replace(',','')) for i in pdx_hood_df['Adjusted population']]

Lets Create a map to show the neighborhoods and thier populations

In [315]:
pdx_map = folium.Map(location=[latitude, longitude], zoom_start=11)
pdx_map.choropleth(
    geo_data=pdx_geo,
    data=pdx_hood_df,
    columns=['Neighborhood', 'Adjusted population'],
    key_on='feature.properties.NAME',
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population in Portland, Oregon',
    reset=True
)
pdx_map

Lets look at the density of each neighborhood

In [316]:
pdx_hood_df['Adjusted population density (people per square mile, excluding parks and industrial tracts)']\
= [int(i.replace(',','')) for i in pdx_hood_df['Adjusted population density (people per square mile, excluding parks and industrial tracts)']]

In [317]:
pdx_map = folium.Map(location=[latitude, longitude], zoom_start=11)
pdx_map.choropleth(
    geo_data=pdx_geo,
    data=pdx_hood_df,
    columns=['Neighborhood', 'Adjusted population density (people per square mile, excluding parks and industrial tracts)'],
    key_on='feature.properties.NAME',
    fill_color='YlGnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population in Portland, Oregon',
    reset=True
)
pdx_map

/home/extralime/.local/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


This is a more interesting map, which conveys more meaningful data. However, it is based on 2018 data that has not been adjusted for new housing

lets correct that, also, lets take what we want from the pdx_hood_df and make a more concise DF

In [476]:
#first lets create a dataframe with basic information about each neighborhood
pdx_df = pdx_hood_df[['Neighborhood','Adjusted population','Median age',
 'Median household income ($)','Walk score','New Units','5-year median price change (2014–2018) (%)','Adjusted population density (people per square mile, excluding parks and industrial tracts)']].copy()

In [477]:
#find the center coords of each neighborhood
lng = []
lat = []
for i in pdx_df['Neighborhood']:
    for j in range(len(pdx_geo['features'])):
        if pdx_geo['features'][j]['properties']['NAME']==i:          
            cs = pdx_geo['features'][j]['geometry']['coordinates'][0]
            if len(cs) == 1:
                lng.append(sum([cs[0][i][0] for i in range(len(cs[0]))])/len(cs[0]))
                lat.append(sum([cs[0][i][1] for i in range(len(cs[0])) ])/len(cs[0]))    
            else: 
                lng.append(sum([cs[i][0] for i in range(len(cs))])/len(cs))
                lat.append(sum([cs[i][1] for i in range(len(cs))])/len(cs))            

In [478]:
#add new data to DF
pdx_df['Latitude'] = lat
pdx_df['Longitude'] = lng

In [480]:
#lets adjust the names of the columns
pdx_df.columns = ['Neighborhood',
 'Adjusted population',
 'Median age',
 'Median household income ',
 'Walk score',
 'New Units',
 '5-year median price change',
 'Adjusted population density ',
 'Latitude',
 'Longitude']

In [491]:
#convert data to mathable types
pdx_df['5-year median price change'][85] = 0
for col in pdx_df.iloc[:,1:]:
    pdx_df[col] = pdx_df[col].astype(str)
    pdx_df[col] = [float(i.replace(',','').replace('%','')) for i in pdx_df[col]]
    

/home/extralime/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [517]:
#Now we will incorporate the the new units, an assumption of 1.5 persons increase per new unit is given.
sq_miles = pd.Series(pdx_df['Adjusted population']/pdx_df['Adjusted population density '])
new_pop = pd.Series(pdx_df['Adjusted population']+ (pdx_df['New Units']*1.5))
pdx_df['new_density']=new_pop/sq_miles
pdx_df['pop_d_change'] = pdx_df['new_density']-pdx_df['Adjusted population density ']

In [521]:
#lets look at the map again and plot the change
pdx_map = folium.Map(location=[latitude, longitude], zoom_start=11)
pdx_map.choropleth(
    geo_data=pdx_geo,
    data=pdx_df,
    columns=['Neighborhood', 'pop_d_change'],
    key_on='feature.properties.NAME',
    fill_color='BuGn', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Expected Population Density Change Portland, Oregon',
    reset=True
)
pdx_map

we need a rough estimate of the neighborhood size in sq meters. to do this we will divide population by population density, this should give us the area in miles, then we will multiply by 1609.4 to give meters

In [494]:
pdx_df['sq_meters'] = (pdx_df['Adjusted population']/pdx_df['Adjusted population density '])*1609.4
pdx_df.head()

,Neighborhood,Adjusted population,Median age,Median household income,Walk score,New Units,5-year median price change,Adjusted population density,Latitude,Longitude,sq_meters
0,ALAMEDA,5641.0,41.0,109616.0,65.0,30.0,26.0,9544.0,45.546252,-122.632071,951.239040
1,ARBOR LODGE,7245.0,36.0,75223.0,72.0,443.0,33.0,8654.0,45.572256,-122.695776,1347.365727
2,ARDENWALD-JOHNSON CREEK,2015.0,41.0,95630.0,54.0,10.0,34.0,10273.0,45.462172,-122.622260,315.676141
3,ARGAY TERRACE,6035.0,40.0,51990.0,45.0,58.0,52.0,6942.0,45.554164,-122.522407,1399.125468
4,ARLINGTON HEIGHTS,1804.0,52.0,113665.0,40.0,1.0,9.0,8934.0,45.517561,-122.708117,324.978464


In [508]:
#lets get a radius for each neighborhood
pdx_df['radius']= np.sqrt(pdx_df['sq_meters']/np.pi)
pdx_df['radius'] = [int(i) for i in pdx_df['radius']]
pdx_df.head()

,Neighborhood,Adjusted population,Median age,Median household income,Walk score,New Units,5-year median price change,Adjusted population density,Latitude,Longitude,sq_meters,radius
0,ALAMEDA,5641.0,41.0,109616.0,65.0,30.0,26.0,9544.0,45.546252,-122.632071,951.239040,17
1,ARBOR LODGE,7245.0,36.0,75223.0,72.0,443.0,33.0,8654.0,45.572256,-122.695776,1347.365727,20
2,ARDENWALD-JOHNSON CREEK,2015.0,41.0,95630.0,54.0,10.0,34.0,10273.0,45.462172,-122.622260,315.676141,10
3,ARGAY TERRACE,6035.0,40.0,51990.0,45.0,58.0,52.0,6942.0,45.554164,-122.522407,1399.125468,21
4,ARLINGTON HEIGHTS,1804.0,52.0,113665.0,40.0,1.0,9.0,8934.0,45.517561,-122.708117,324.978464,10


In [510]:
pdx_df

,Neighborhood,Adjusted population,Median age,Median household income,Walk score,New Units,5-year median price change,Adjusted population density,Latitude,Longitude,sq_meters,radius,new_density
0,ALAMEDA,5641.0,41.0,109616.0,65.0,30.0,26.0,9544.0,45.546252,-122.632071,951.239040,17,9620.135437
1,ARBOR LODGE,7245.0,36.0,75223.0,72.0,443.0,33.0,8654.0,45.572256,-122.695776,1347.365727,20,9447.731263
2,ARDENWALD-JOHNSON CREEK,2015.0,41.0,95630.0,54.0,10.0,34.0,10273.0,45.462172,-122.622260,315.676141,10,10349.473945
3,ARGAY TERRACE,6035.0,40.0,51990.0,45.0,58.0,52.0,6942.0,45.554164,-122.522407,1399.125468,21,7042.075228
4,ARLINGTON HEIGHTS,1804.0,52.0,113665.0,40.0,1.0,9.0,8934.0,45.517561,-122.708117,324.978464,10,8941.428492
5,ARNOLD CREEK,2963.0,43.0,134282.0,17.0,28.0,55.0,2877.0,45.440326,-122.699606,1657.508585,22,2917.780965
6,ASHCREEK,4763.0,39.0,92676.0,36.0,26.0,32.0,4129.0,45.464887,-122.746927,1856.520271,24,4162.808734
7,BEAUMONT-WILSHIRE,6124.0,41.0,112725.0,72.0,105.0,27.0,8388.0,45.546483,-122.620322,1175.007821,19,8603.726649
8,BOISE,4430.0,34.0,75505.0,92.0,680.0,25.0,12027.0,45.549191,-122.671485,592.803027,13,14796.196388
9,BRENTWOOD-DARLINGTON,12823.0,37.0,59388.0,54.0,151.0,58.0,8027.0,45.462326,-122.597163,2570.989934,28,8168.785503


In [495]:
#look at a map with the neighborhoods
# create map of Toronto using latitude and longitude values from geocoder
pn_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(pdx_df['Latitude'], pdx_df['Longitude'], pdx_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(pn_map)  
    
pn_map

Now we can use foursquare to get a list of venues in each neighborhood

In [496]:
#this cell retrieves id and key info for the foursquare api
with open('foursquare.json') as f:
    fs = f.read()
creds = json.loads(fs)
CLIENT_ID = creds['CLIENT_ID']
CLIENT_SECRET = creds['CLIENT_SECRET']
VERSION = creds["VERSION"]

In [548]:
pdx_df[pdx_df['Neighborhood']=='PORTLAND DOWNTOWN']

,Neighborhood,Adjusted population,Median age,Median household income,Walk score,New Units,5-year median price change,Adjusted population density,Latitude,Longitude,sq_meters,radius,new_density,pop_d_change
21,PORTLAND DOWNTOWN,11613.0,40.0,33634.0,95.0,1473.0,31.0,15119.0,45.514221,-122.680271,1236.19037,19,17995.554766,2876.554766


In [506]:
pdx_df.loc[0]['Latitude']

45.546251733485825

In [549]:
#Lets check out some venues in ALAMEDA, using the center coords and since portland is a
#walking city we can use a radius of 1500
LIMIT=1000
radius=1500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    pdx_df.loc[21]['Latitude'], 
    pdx_df.loc[21]['Longitude'], 
    radius, 
    LIMIT)

In [550]:
results = requests.get(url).json()

In [551]:
len(results['response']['groups'][0]['items'])

100

In [525]:
#Borrow function from foursquare lab
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [526]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Guilder,Coffee Shop,45.548290,-122.641335
1,Grant Park,Park,45.539932,-122.629707
2,Garden Fever,Garden Center,45.548010,-122.641359
3,Lucca,Italian Restaurant,45.548167,-122.641375
4,Little Griddle,Breakfast Spot,45.548591,-122.620037


In [527]:
#lets see how many venues were retireved
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.
